<a href="https://colab.research.google.com/github/soheilpaper/-tft-2.4-ili9341-STM32/blob/master/youtube_subtitle/ASR_%26_Speaker_Diarization_with_OpenAI_Whisper_%26_Nvidia_Nemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ASR (OpenAI Whisper)

In this section, we'll use OpenAI newly release Whisper to convert a sample youtube podcast to words <> timestamps mapping.

In [ ]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [ ]:
!pip3 install torch torchvision torchaudio yt-dlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 KB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 12.3 MB/s eta 0:00:00


In [ ]:
# A 20mins podcast from YC official youtube channel.
!rm ./audio.wav
!yt-dlp -xv --audio-format wav  -o audio.wav -- https://www.youtube.com/watch?v=bp_kMA-eTsE

rm: cannot remove './audio.wav': No such file or directory
[debug] Command-line config: ['-xv', '--audio-format', 'wav', '-o', 'audio.wav', '--', 'https://www.youtube.com/watch?v=bp_kMA-eTsE']
[debug] Encodings: locale UTF-8, fs utf-8, pref UTF-8, out utf-8, error utf-8, screen utf-8
[debug] yt-dlp version 2023.01.06 [6becd25] (pip)
[debug] Python 3.8.16 (CPython x86_64 64bit) - Linux-5.10.147+-x86_64-with-glibc2.27 (OpenSSL 1.1.1  11 Sep 2018, glibc 2.27)
[debug] exe versions: ffmpeg 3.4.11, ffprobe 3.4.11
[debug] Optional libraries: Cryptodome-3.16.0, brotli-1.0.9, certifi-2022.12.07, mutagen-1.46.0, sqlite3-2.6.0, websockets-10.4
[debug] Proxy map: {}
[debug] Loaded 1760 extractors
[youtube] Extracting URL: https://www.youtube.com/watch?v=bp_kMA-eTsE
[youtube] bp_kMA-eTsE: Downloading webpage
[youtube] bp_kMA-eTsE: Downloading android player API JSON
[debug] Sort order given by extractor: quality, res, fps, hdr:12, source, vcodec:vp9.2, channels, acodec, lang, proto
[debug] Formats 

In [ ]:
!rm ./audio_16k.wav
!ffmpeg -i audio.wav -ac 1 -ar 16000 audio_16k.wav # Converting audio.wav to mono channel & 16K audio_16k.wav

rm: cannot remove './audio_16k.wav': No such file or directory
ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp

In [ ]:
# Installing Whisper and WhisperX

!pip install git+https://github.com/openai/whisper.git
!pip install git+https://github.com/m-bain/whisperX.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-cw37414t
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-cw37414t
  Resolved https://github.com/openai/whisper.git to commit f82bc59f5ea234d4b97fb2860842ed38519f7e65
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 19.4 MB/s eta 0:00:00
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175340 sha256=884d09d0a33dcb78b89b347f4a91a1f7f43f92ae62bb519134896a195f2c75c0
  Stored in directory: /tmp/pip-ephem-wheel-cache-3wisro1c/wheels/a7/70/18/b7693c07b1d18b3dafb328f5d0496aa0d41a9c09ef332fd8e6
Successfully built whisper
Looking in index

In [ ]:
from whisper import load_model

# Large models result in considerably better and more aligned (words, timestamps) mapping.
model = load_model("large-v2")

# Beam size if None by default (Greedy Decoding). You can also set the
# beam_size to some number like 5. This will increase in better transcription
# quality but it'll increase runtime considerabley.
results = model.transcribe('./audio_16k.wav', beam_size=None)

100%|█████████████████████████████████████| 2.87G/2.87G [00:47<00:00, 65.2MiB/s]


In [ ]:
# WhisperX results in better word timestamps by using wav2vec based forced alignment.
import whisperx

device = 'cuda'
alignment_model, metadata = whisperx.load_align_model(language_code=results["language"], device=device)
result_aligned = whisperx.align(results["segments"], alignment_model, metadata, './audio_16k.wav', device)

Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960_asr_ls960.pth


  0%|          | 0.00/360M [00:00<?, ?B/s]

[00:00.281 --> 00:04.138]  The first time founders can actually take more risk on the ideas that they pick because
[00:04.701 --> 00:08.520]  they don't have other startup friends or they don't care as much.
[00:08.600 --> 00:10.158]  They're just working on stuff they find interesting.
[00:10.402 --> 00:10.851]  I love that.
[00:11.340 --> 00:12.186]  They have nobody to impress.
[00:14.461 --> 00:15.947]  Yeah, basically.
[00:19.846 --> 00:21.896]  Hello, this is Michael with Harge and Brad.
[00:22.680 --> 00:24.655]  Welcome to Inside the Group Partner Lounge.
[00:25.240 --> 00:29.579]  As YC Group Partners, we often find ourselves repeating the same advice over and over again
[00:29.920 --> 00:31.073]  to start a founders week real quick.
[00:31.680 --> 00:34.980]  Before COVID, we'd often gather together in the Group Partner Lounge at the YC office
[00:35.201 --> 00:38.978]  to try to figure out why this was the case and how we can help startups figure it out
[00:39.320 --> 00:39.

You WhisperX alignment may fail. If this happens then it's most probably because Whisper just `hallucinated` i.e. whisper came up with extra/weird output at the end.

This usually happens with long audio files. If this happens, I'd suggest splitting big audio files in small files.

In [ ]:
# Storing words <> timestamps mapping in a file.
import json

with open('./word_ts.text', 'w+') as f:
    for line in result_aligned['word_segments']:
        line_temp = line.copy()
        # WhisperX don't put a space after word but just to make sure.
        line_temp['text'] = line_temp['text'].strip()
        f.write(f'{json.dumps(line_temp)}\n')

# Speaker Diarization (Nvidian NEMO)

In this section, we'll find timestamps <> Speaker Labels mapping using Nvidia NEMO.

In [ ]:
!apt install sox libsndfile1

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 21 not upgraded.
Need to get 760 kB of archives.
After this operation, 6,717 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopencore-amrnb0 amd64 0.1.3-2.1 [92.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopencore-

In [ ]:
!pip install --upgrade hydra-core llvmlite omegaconf --ignore-installed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.2/701.2 KB 40.5 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=b0c7324f4da6d58c5c05f28dabc529ef045851a4a43de14db198ec941ad18ae9
  Stored in directory: /root/.cache/pip/wheels/b1/a3/c2/6df046c09459b73cc9bb6c4401b0be6c47048baf9a1617c485
Successfully built antlr4-python3-runtime
ERROR: pip's dependency resolver does not c

In [ ]:
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@main

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-req-build-dsingtbh
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo.git /tmp/pip-req-build-dsingtbh
  Resolved https://github.com/NVIDIA/NeMo.git to commit dd921e792427e3788ad9fe3a1e568afc85a8bc93
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 KB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install --upgrade Cython jiwer braceexpand webdataset librosa sentencepiece
!pip install --upgrade youtokentome pyannote-audio transformers pandas inflect editdistance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 KB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 38.6 MB/s eta 0:00:00
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.32
    Uninstalling Cython-0.29.32:
      Successfully uninstalled Cython-0.29.32
  Attempting uninstall: librosa
    Found existing installation: librosa 0.8.1
    Uninstalling librosa-0.8.1:
      Successfully uninstalled librosa-0.8.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━

In [ ]:
!pip install -U pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 KB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 17.4 MB/s eta 0:00:00
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.6.5
    Uninstalling pytorch-lightning-1.6.5:
      Successfully uninstalled pytorch-lightning-1.6.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyannote-audio 2.1.1 requires pytorch-lightning<1.7,>=1.5.4, but you have pytorch-lightning 1.8.6 which is incompatible.


You may have to restart the runtime at this stage for pytorch-lightning to take effect. Otherwise you may see errors like `Couldn't find lightning.core or lightining.logging`.

In [ ]:
import os
import json

diarize_manifest = {
  'audio_filepath': f'./audio_16k.wav',
  'offset': 0,
  'duration':  None,
  'label': "infer",
  'text': "-",
  'num_speakers': None,
  'rttm_filepath': f'./diarized/pred_rttms/audio_16k.rttm',
  'uniq_id': ""
}

if not os.path.exists('./manifest.json'):
  with open('./manifest.json', 'w') as f:
    f.write(json.dumps(diarize_manifest))

In [ ]:
import os
import wget
from omegaconf import OmegaConf
MODEL_CONFIG = os.path.join('./','diar_infer_meeting.yaml')
if not os.path.exists(MODEL_CONFIG):
    config_url = "https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/speaker_tasks/diarization/conf/inference/diar_infer_meeting.yaml"
    MODEL_CONFIG = wget.download(config_url, './')

config = OmegaConf.load(MODEL_CONFIG)

In [ ]:
config.num_workers = 4
config.batch_size = 32

config.diarizer.manifest_filepath = './manifest.json'
config.diarizer.out_dir = os.path.join('./', 'diarized')
config.diarizer.speaker_embeddings.model_path = 'titanet_large'
config.diarizer.speaker_embeddings.parameters.window_length_in_sec = [1.5, 1.0, 0.5]
config.diarizer.speaker_embeddings.parameters.shift_length_in_sec = [0.75, 0.5, 0.25]
config.diarizer.speaker_embeddings.parameters.multiscale_weights = [0.33, 0.33, 0.33]
config.diarizer.speaker_embeddings.parameters.save_embeddings = False

config.diarizer.ignore_overlap = False
config.diarizer.oracle_vad = False
config.diarizer.collar = 0.25


config.diarizer.vad.model_path = 'vad_multilingual_marblenet'
config.diarizer.oracle_vad = False # ----> Not using oracle VAD

In [ ]:
from nemo.collections.asr.models.msdd_models import ClusteringDiarizer

model = ClusteringDiarizer(cfg=config)

[NeMo W 2023-01-15 05:10:43 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-01-15 05:10:43 experimental:27] Module <class 'nemo.collections.asr.models.audio_to_audio_model.AudioToAudioModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-01-15 05:10:46 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-01-15 05:10:46 nemo_logging:349] /usr/local/lib/python3.8/dist-packages/torch/jit/annotations.py:299: UserWarning: TorchScript will treat type annotations of Tensor dtype-specific subtypes as if they are normal Tensors. dtype constraints are not enforced in compilation either.
      warnings.warn("TorchScript will treat type annotations of Tensor "
    
[NeMo W 2023-01-15 05:10:47 experimental:27] Module <class 'n

[NeMo I 2023-01-15 05:10:47 clustering_diarizer:130] Loading pretrained vad_multilingual_marblenet model from NGC
[NeMo I 2023-01-15 05:10:47 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/vad_multilingual_marblenet/versions/1.10.0/files/vad_multilingual_marblenet.nemo to /root/.cache/torch/NeMo/NeMo_1.15.0rc0/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo
[NeMo I 2023-01-15 05:10:48 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-01-15 05:10:48 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/ami_train_0.63.json,/manifests/freesound_background_train.json,/manifests/freesound_laughter_train.json,/manifests/fisher_2004_background.json,/manifests/fisher_2004_speech_sampled.json,/manifests/google_train_manifest.json,/manifests/icsi_all_0.63.json,/manifests/musan_freesound_train.json,/manifests/musan_music_train.json,/manifests/musan_soundbible_train.json,/manifests/mandarin_train_sample.json,/manifests/german_train_sample.json,/manifests/spanish_train_sample.json,/manifests/french_train_sample.json,/manifests/russian_train_sample.json
    sample_rate: 16000
    labels:
    - background
    - speech
    batch_size: 256
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: sca

[NeMo I 2023-01-15 05:10:48 features:267] PADDING: 16
[NeMo I 2023-01-15 05:10:49 save_restore_connector:243] Model EncDecClassificationModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.15.0rc0/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo.
[NeMo I 2023-01-15 05:10:49 clustering_diarizer:157] Loading pretrained titanet_large model from NGC
[NeMo I 2023-01-15 05:10:49 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/titanet_large/versions/v0/files/titanet-l.nemo to /root/.cache/torch/NeMo/NeMo_1.15.0rc0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-01-15 05:10:51 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-01-15 05:10:53 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-01-15 05:10:53 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-01-15 05:10:53 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-01-15 05:10:53 features:267] PADDING: 16
[NeMo I 2023-01-15 05:10:54 audio_preprocessing:513] Numba CUDA SpecAugment kernel is being used
[NeMo I 2023-01-15 05:10:54 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.15.0rc0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


In [ ]:
model.diarize()

[NeMo I 2023-01-15 05:10:54 speaker_utils:92] Number of files to diarize: 1
[NeMo I 2023-01-15 05:10:54 clustering_diarizer:304] Split long audio file to avoid CUDA memory issue


splitting manifest: 100%|██████████| 1/1 [00:00<00:00,  1.43it/s]

[NeMo I 2023-01-15 05:10:56 classification_models:263] Perform streaming frame-level VAD


[NeMo I 2023-01-15 05:10:56 collections:298] Filtered duration for loading collection is 0.000000.
[NeMo I 2023-01-15 05:10:56 collections:301] Dataset loaded with 27 items, total duration of  0.37 hours.
[NeMo I 2023-01-15 05:10:56 collections:303] # 27 files loaded accounting to # 1 labels


[NeMo W 2023-01-15 05:10:56 nemo_logging:349] /usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      warnings.warn(_create_warning_msg(
    
vad: 100%|██████████| 27/27 [00:11<00:00,  2.31it/s]

...

[2/3] extract embeddings: 100%|██████████| 80/80 [00:04<00:00, 19.79it/s]


[NeMo I 2023-01-15 05:11:17 clustering_diarizer:282] Subsegmentation for embedding extraction: scale2, ./diarized/speaker_outputs/subsegments_scale2.json
[NeMo I 2023-01-15 05:11:18 clustering_diarizer:337] Extracting embeddings for Diarization
[NeMo I 2023-01-15 05:11:18 collections:298] Filtered duration for loading collection is 0.000000.
[NeMo I 2023-01-15 05:11:18 collections:301] Dataset loaded with 5099 items, total duration of  0.71 hours.
[NeMo I 2023-01-15 05:11:18 collections:303] # 5099 files loaded accounting to # 1 labels


clustering: 100%|██████████| 1/1 [00:17<00:00, 17.49s/it]
[NeMo W 2023-01-15 05:11:43 nemo_logging:349] /usr/local/lib/python3.8/dist-packages/pyannote/metrics/utils.py:200: UserWarning: 'uem' was approximated by the union of 'reference' and 'hypothesis' extents.
      warnings.warn(
    


[NeMo I 2023-01-15 05:11:43 der:96] Cumulative Results for collar 0.25 sec and ignore_overlap False: 
     FA: 0.0000	 MISS 0.0000	                 Diarization ER: 0.0000	, Confusion ER:0.0000
[NeMo I 2023-01-15 05:11:43 clustering_diarizer:456] Outputs are saved in /content/diarized directory


(<pyannote.metrics.diarization.DiarizationErrorRate at 0x7f83b1d75bb0>,
 {'audio_16k': {'speaker_0': 'speaker_0',
   'speaker_1': 'speaker_1',
   'speaker_2': 'speaker_2'}},
 (0.0, 0.0, 0.0, 0.0))

In [ ]:
# Reading timestamps <> Speaker Labels mapping

speaker_ts = []
with open('./diarized/pred_rttms/audio_16k.rttm', 'r') as f:
    lines = f.readlines()
    for line in lines:
        line_list = line.split(' ')
        s = int(float(line_list[5]) * 1000)
        e = s + int(float(line_list[8]) * 1000)
        speaker_ts.append([s, e, int(line_list[11].split('_')[-1])])


# Post-processing

From previous cells, we have got `word_ts` (words <> timestamps mapping) and `speaker_ts` (speaker label <> timestamps mapping). In this cell, we are going
to merge them together to get a list of (word, speaker label, timestamps) tuples.

In [ ]:
# Reading words <> timestamps mapping, which we saved earlier
import json

word_ts = []
with open('./word_ts.text', 'r+') as f:
    for line in f:
        line_temp = json.loads(line)
        word_ts.append(line_temp)

In [ ]:
def get_word_ts_anchor(s, e, option='start'):
  if option == 'end':
    return e
  elif option == 'mid':
    return (s + e) / 2
  return s

def get_words_speaker_mapping(wrd_ts, spk_ts, word_anchor_option='start'):
    s, e, sp = spk_ts[0]
    wrd_pos, turn_idx = 0, 0
    wrd_spk_mapping = []
    for wrd_dict in wrd_ts:
        ws, we, wrd = int(wrd_dict['start'] * 1000), int(wrd_dict['end'] * 1000), wrd_dict['text']
        wrd_pos = get_word_ts_anchor(ws, we, word_anchor_option)
        while wrd_pos > float(e):
            turn_idx += 1
            turn_idx = min(turn_idx, len(spk_ts) - 1)
            s, e, sp = spk_ts[turn_idx]
        wrd_spk_mapping.append({'word': wrd, 'start_time': ws, 'end_time': we, 'speaker': sp})
    return wrd_spk_mapping

In [ ]:
wsm = get_words_speaker_mapping(word_ts, speaker_ts, 'start')

To get accurate diarization results, we rely on correct sentence-ending punctuation i.e. period(.), question mark(?) and, exclamation(!). In all the test-cases, which I have performed using this diarization technique, Whisper was able to come up with correct punctuation markings. But there are chances for whisper to get stuck in no-punctuation loop.

One way to fix this is to use a neutral initial prompt with punctuations e.g. `model.transcribe(FILE_NAME, initial_promot='How are you doing? I hope everything is OK.')`. This approach has couple of disadvantages:



*   It could affect the transcription quality e.g. for one of my test cases, using `initial_promot` gave me `founders week real quick`, whereas the ground truth was `founders we'd work with.`
*   Transcription timestamps could be out-of-sync. For one of the examples, I observed a lag of 2 seconds which could be fixed only if we knew beforehand how much that'd be.

A better approach is to use a transformer based model to restore punctuation of the transcript.



In [ ]:
!pip install deepmultilingualpunctuation

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from deepmultilingualpunctuation import PunctuationModel

punct_model = PunctuationModel()
words_list = list(map(lambda x: x['word'], wsm))

labled_words = punct_model.predict(words_list)

Downloading:   0%|          | 0.00/892 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

[NeMo W 2023-01-15 05:12:50 nemo_logging:349] /usr/local/lib/python3.8/dist-packages/transformers/pipelines/token_classification.py:159: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="none"` instead.
      warnings.warn(
    
[NeMo W 2023-01-15 05:12:51 nemo_logging:349] /usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
      warnings.warn(
    


In [ ]:
# Whisper already punctuates the text in most of the case, so we'll give priority
# to its puntuation marks over PunctuationModel results.
import re

ending_puncts = '.?!'
model_puncts = '.,;:!?'

# We don't want to punctuate U.S.A. with a period. Right?
is_acronym = lambda x: re.fullmatch(r"\b(?:[a-zA-Z]\.){2,}", x)

for word_dict, labeled_tuple in zip(wsm, labled_words):
    word = word_dict['word']
    if word and labeled_tuple[1] in ending_puncts and (word[-1] not in model_puncts or is_acronym(word)):
        word += labeled_tuple[1]
        if word.endswith('..'): word = word.rstrip('.')
        word_dict['word'] = word

# Realignment using N-Gram/Masked Language Models

In previous cells, we were able to get (word, timestamps, speaker_label) tuples which are also properly punctuated. Now we are in position to build a diarized transcription. Something like following:

```
Speaker A: Hey What's up?
Speaker B: Yeah everything is good. How about you?
.....
```

Wihtout doing any pre-processing, we'll see following results more than often.

```
Speaker A: It's got to come from somewhere else. Yeah, that one's also fun because you know the lows are
Speaker B: going to suck, right? So it's actually it hits you on both sides.
.....
```

Or something like:

```
Speaker A: That was an amazon podcast. Oh
Speaker B: yeah, I really enjoyed it.
.....
```

So you will see a splitted sentence on change of speaker. This could happen due to multiple reasons e.g. Overlapped speech, background noise or faulty timestamps (either from ASR or Diarization model)

There are couple of ways to fix that. One simple way is to use an n-gram LM (KenLM) or a masked LM to calculate sentence perplexity (Log probability). To do that, we'll get last few words of Speaker A utterance and first few words of Speaker B utterences, add `\n` at different places and calculate the sentence perplexity. Optimal position of `\n` would be where probability is maximum. Something like following:

```
Speaker A: That was an amazon
Speaker B: podcast. Oh yeah, I really enjoyed it.
perplexity_score: 1

Speaker A: That was an amazon podcast.
Speaker B: Oh yeah, I really enjoyed it.
perplexity_score: 5

Speaker A: That was an amazon podcast. Oh
Speaker B: yeah, I really enjoyed it.
perplexity_score: 3

Speaker A: That was an amazon podcast. Oh yeah.
Speaker B: I really enjoyed it.
perplexity_score: 4

Speaker A: That was an amazon podcast. Oh yeah. I
Speaker B: really enjoyed it.
perplexity_score: 1
.....
```

I tried this approach on multiple examples but results were not consistent.


# Realignment using Punctuations

Another simple method (You can call it a hack) is to using puctuation markings. If a sentence is splitted between two different speakers, it can either be part of Speaker A utterance or Speaker B utterance. We cans simply take mode of speaker labels and use that speaker label for whole sentence. Something like following:

```
Speaker A: It's got to come from somewhere else. Yeah, that one's also fun because you know the lows are
Speaker B: going to suck, right? So it's actually it hits you on both sides.
```

Speaker A has more part of the sentence `Yeah, that one's also ..... right?` so this whole sentence should be part of Speaker A utterance.

This can gets interesting when one speaker is saying some sort of monologue and other is doing `Hmm`, `Yeah`, `Exactly` in background. Whisper will tend to ignore that but Diarization Model (Nemo) will pick it up. So you may get inconsistent outputs like following:

```
Speaker A: It's got to come from somewhere
Speaker B: else. Yeah,
Speaker A: that one's also fun because you know the
Speaker C: lows are
Speaker A: going to suck, right?
```

So only Speaker A was speaking. Other's `hmm`, `yeah` were ignored by Whisper but because Diarization model was able to give us timestamps for them, we saw this inconsistent output.

Fortunately, our punctuation based realignment will fix that.

There are some special cases which are hard to fix i.e. when we have equal part of both sentences splitted between two Speakers. Something like

```
Speaker A: Oh
Speaker B: Yeah.
Speaker A: Yeah
Speaker B: Exactly.
```

Unfortunately these can't be fixed and we have to accept them as part of diarization error. Our realignment code will assign them to either of the speaker.

In [ ]:
sentence_ending_punctuations = '.?!'

def get_first_word_idx_of_sentence(word_idx, word_list, speaker_list, max_words):
  is_word_sentence_end = lambda x: x >= 0 and word_list[x][-1] in sentence_ending_punctuations
  left_idx = word_idx
  while (left_idx > 0 and word_idx - left_idx < max_words and
          speaker_list[left_idx - 1] == speaker_list[left_idx] and
          not is_word_sentence_end(left_idx - 1)):
      left_idx -= 1

  return left_idx if left_idx == 0 or is_word_sentence_end(left_idx - 1) else -1

def get_last_word_idx_of_sentence(word_idx, word_list, max_words):
  is_word_sentence_end = lambda x: x >= 0 and word_list[x][-1] in sentence_ending_punctuations
  right_idx = word_idx
  while (right_idx < len(word_list) and right_idx - word_idx < max_words and
          not is_word_sentence_end(right_idx)):
      right_idx += 1

  return right_idx if right_idx == len(word_list) - 1 or is_word_sentence_end(right_idx) else -1

def get_realigned_ws_mapping_with_punctuation(word_speaker_mapping, max_words_in_sentence = 50):
  is_word_sentence_end = lambda x: x >= 0 and word_speaker_mapping[x]['word'][-1] in sentence_ending_punctuations
  wsp_len = len(word_speaker_mapping)

  words_list, speaker_list = [], []
  for k, line_dict in enumerate(word_speaker_mapping):
      word, speaker = line_dict['word'], line_dict['speaker']
      words_list.append(word)
      speaker_list.append(speaker)

  k = 0
  while k < len(word_speaker_mapping):
      line_dict = word_speaker_mapping[k]
      if k < wsp_len - 1 and speaker_list[k] != speaker_list[k + 1] and not is_word_sentence_end(k):
          left_idx = get_first_word_idx_of_sentence(k, words_list, speaker_list, max_words_in_sentence)
          right_idx = get_last_word_idx_of_sentence(k, words_list, max_words_in_sentence - k + left_idx - 1) if left_idx > -1 else -1
          if min(left_idx, right_idx) == -1:
              k += 1
              continue

          spk_labels = speaker_list[left_idx: right_idx + 1]
          mod_speaker = max(set(spk_labels), key=spk_labels.count)
          if spk_labels.count(mod_speaker) < len(spk_labels) // 2:
              k += 1
              continue

          speaker_list[left_idx: right_idx + 1] = [mod_speaker] * (right_idx - left_idx + 1)
          k = right_idx

      k += 1

  k, realigned_list = 0, []
  while k < len(word_speaker_mapping):
      line_dict = word_speaker_mapping[k].copy()
      line_dict['speaker'] = speaker_list[k]
      realigned_list.append(line_dict)
      k += 1


  return realigned_list

In [ ]:
def get_sentences_speaker_mapping(word_speaker_mapping, spk_ts):
  s, e, spk = spk_ts[0]
  prev_spk = spk

  snts = []
  snt = {'speaker': f'Speaker {spk}', 'start_time': s, 'end_time': e, 'text': ''}

  for wrd_dict in word_speaker_mapping:
      wrd, spk = wrd_dict['word'], wrd_dict['speaker']
      s, e = wrd_dict['start_time'], wrd_dict['end_time']
      if spk != prev_spk:
          snts.append(snt)
          snt = {'speaker': f'Speaker {spk}', 'start_time': s, 'end_time': e, 'text': ''}
      else:
          snt['end_time'] = e
      snt['text'] += wrd + ' '
      prev_spk = spk

  snts.append(snt)
  return snts

def get_speaker_aware_transcript(sentences_speaker_mapping):
  with open('diarization.txt', 'w') as f:
    for sentence_dict in sentences_speaker_mapping:
        sp = sentence_dict['speaker']
        text = sentence_dict['text']
        f.write(f'\n\n{sp}: {text}')

In [ ]:
wsm = get_realigned_ws_mapping_with_punctuation(wsm)
ssm = get_sentences_speaker_mapping(wsm, speaker_ts)
get_speaker_aware_transcript(ssm)